## Building the example for Versa ECP5 board

Check out the sources, if not existing:

In [1]:
! [ -e $HOME/work/ghdl-yosys-plugin ] || ( cd $HOME/work && git clone https://github.com/ghdl/ghdl-yosys-plugin.git )

In [2]:
%cd /home/pyosys/work/ghdl-yosys-plugin/examples/ecp5_versa 

/home/pyosys/work/ghdl-yosys-plugin/examples/ecp5_versa


In [3]:
!make clean GHDL=ghdl YOSYS="yosys -q" versa_ecp5_top.json

rm -fr lib work *.json *.svf *.config *-report.txt
mkdir lib
ghdl -i --workdir=lib/ --work=ecp5um ../../library/ecp5u/components.vhdl
yosys -q -m ghdl -p \
	"ghdl -Plib -gCLK_FREQUENCY=25000000 versa_ecp5_top.vhdl pll_mac.vhd soc_iomap_pkg.vhdl uart.vhdl uart_tx.vhdl uart_rx.vhdl fifobuf.vhdl -e versa_ecp5_top; \
	read_verilog ../../library/wrapper/primitives.v ../../library/wrapper/bram.v; \
	synth_ecp5 \
	-top versa_ecp5_top -json versa_ecp5_top.json" 2>&1 | tee versa_ecp5_top-report.txt
pll_mac.vhd:53:5:warning: instance "scuba_vhi_inst" of component "vhi" is not bound [-Wbinding]
pll_mac.vhd:22:14:warning: (in default configuration of pll_mac(structure))
pll_mac.vhd:56:5:warning: instance "scuba_vlo_inst" of component "vlo" is not bound [-Wbinding]
pll_mac.vhd:22:14:warning: (in default configuration of pll_mac(structure))
pll_mac.vhd:59:5:warning: instance "pllinst_0" of component "ehxplll" is not bound [-Wbinding]
pll_mac.vhd:22:14:warning: (in default configuration of pll_mac(st

## Place & route

Install the required packages:

In [4]:
! sudo apt-get install -y nextpnr-ecp5 libtrellis

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libboost-atomic1.67.0 libboost-chrono1.67.0 libboost-date-time1.67.0
  libboost-iostreams1.67.0 libboost-program-options1.67.0 libboost-regex1.67.0
  libboost-thread1.67.0
The following NEW packages will be installed:
  libboost-atomic1.67.0 libboost-chrono1.67.0 libboost-date-time1.67.0
  libboost-iostreams1.67.0 libboost-program-options1.67.0 libboost-regex1.67.0
  libboost-thread1.67.0 libtrellis nextpnr-ecp5
0 upgraded, 9 newly installed, 0 to remove and 3 not upgraded.
Need to get 8072 kB of archives.
After this operation, 204 MB of additional disk space will be used.
Get:2 http://deb.debian.org/debian buster/main amd64 libboost-atomic1.67.0 amd64 1.67.0-13+deb10u1 [226 kB]
Get:4 http://deb.debian.org/debian buster/main amd64 libboost-chrono1.67.0 amd64 1.67.0-13+deb10u1 [234 kB]
Get:5 http://deb.debian.org/debian buster/main amd64 

In [5]:
! make GHDL=ghdl YOSYS="yosys -Q" NEXTPNR=nextpnr-ecp5 ECPPACK=ecppack

ghdl -i --workdir=lib/ --work=ecp5um ../../library/ecp5u/components.vhdl
nextpnr-ecp5 --json versa_ecp5_top.json --lpf versa_ecp5.lpf \
	--textcfg versa_ecp5_top.config --um5g-45k --freq 100 --lpf-allow-unconstrained --package CABGA381
Info: constraining clock net 'clk_in' to 100.00 MHz

Info: Logic utilisation before packing:
Info:     Total LUT4s:       399/43848     0%
Info:         logic LUTs:    223/43848     0%
Info:         carry LUTs:     80/43848     0%
Info:           RAM LUTs:     64/21924     0%
Info:          RAMW LUTs:     32/10962     0%

Info:      Total DFFs:       362/43848     0%

Info: Packing IOs..
Info: pin 'clk_in$tr_io' constrained to Bel 'X0/Y68/PIOC'.
Info: pin 'reset_n$tr_io' constrained to Bel 'X4/Y71/PIOB'.
Info: pin 'rxd_uart$tr_io' constrained to Bel 'X38/Y0/PIOB'.
Info: pin 'segdp$tr_io' constrained to Bel 'X6/Y71/PIOA'.
Info: $twi_scl$iobuf_i: twi_scl$const$VCC$609.Y
Info: $twi_sda$iobuf_i: twi_sda$const$VCC$610.Y
Info: pin 'txd_uart$tr_io' constrained 

Info: Routing globals...
Info:     routing clock net $glbnet$mclk using global 0

Info: Routing..
Info: Setting up routing queue.
Info: Routing 1811 arcs.
Info:            |   (re-)routed arcs  |   delta    | remaining|       time spent     |
Info:    IterCnt |  w/ripup   wo/ripup |  w/r  wo/r |      arcs| batch(sec) total(sec)|
Info:       1000 |        5        994 |    5   994 |       817|       0.34       0.34|
Info:       1825 |       12       1813 |    7   819 |         0|       0.32       0.67|
Info: Routing complete.
Info: Router1 time 0.67s
Info: Checksum: 0x5b8c60b6

Info: Critical path report for clock '$glbnet$mclk' (posedge -> negedge):
Info: curr total
Info:  0.3  0.3  Source uart_inst.txfifo.optr_TRELLIS_FF_DI_1_SLICE.Q0
Info:  1.2  1.5    Net uart_inst.txfifo.ram.mem.0.0.0_RAD[2] budget 2.500000 ns (17,3) -> (20,4)
Info:                Sink uart_inst.txfifo.ram.mem.1.0.0$DPRAM1_SLICE.C0
Info:  0.1  1.7  Source uart_inst.txfifo.ram.mem.1.0.0$DPRAM1_SLICE.F0
Info:  0.7  2

### Programming the target

In [6]:
! sudo apt-get install -y openocd

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libftdi1-2 libhidapi-hidraw0 libjaylink0 libjim0.77 libusb-0.1-4
  libusb-1.0-0
The following NEW packages will be installed:
  libftdi1-2 libhidapi-hidraw0 libjaylink0 libjim0.77 libusb-0.1-4
  libusb-1.0-0 openocd
0 upgraded, 7 newly installed, 0 to remove and 3 not upgraded.
Need to get 2710 kB of archives.
After this operation, 6939 kB of additional disk space will be used.
Get:1 http://deb.debian.org/debian buster/main amd64 libusb-1.0-0 amd64 2:1.0.22-2 [55.3 kB]
Get:2 http://deb.debian.org/debian buster/main amd64 libftdi1-2 amd64 1.4-1+b2 [30.2 kB]
Get:3 http://deb.debian.org/debian buster/main amd64 libhidapi-hidraw0 amd64 0.8.0~rc1+git20140818.d17db57+dfsg-2 [10.4 kB]
Get:4 http://deb.debian.org/debian buster/main amd64 libjaylink0 amd64 0.1.0-1 [17.1 kB]
Get:5 http://deb.debian.org/debian buster/main amd64 libusb-0.1-4 amd64 2

## Download to target

In order to access the USB device, the container must be run with the `--device=/dev/bus/usb` option.

In [7]:
! sudo openocd -f ../../openocd/ecp5-versa.cfg -f ../../openocd/LFE5UM5G-45F.cfg \
    -c "transport select jtag; init; svf versa_ecp5_top.svf; exit"

Open On-Chip Debugger 0.10.0
Licensed under GNU GPL v2
For bug reports, read
	http://openocd.org/doc/doxygen/bugs.html
none separate
adapter speed: 5000 kHz
Info : auto-selecting first available session transport "jtag". To override use 'transport select <transport>'.
Warn : Transport "jtag" was already selected
Error: libusb_open() failed with LIBUSB_ERROR_NO_DEVICE
Error: no device found
Error: unable to open ftdi device with vid 0403, pid 6010, description '*', serial '*' at bus location '*'



The orange LED should now be blinking.

### Testing the UART loopback

**UART loopback test omitted due to missing packages**